# Task 1 f, Calculating reserve factors against column buckling using Euler-Johnson

## We need from the data file:
- volume of the element
- Iyy calculated with `height_str, width_str, thickness_web, thickness_flange, thickness_skin, stringer_pitch`
- dimensions
- `EulerJohnson(EModulus, I_y, area, length, height_str, thickness_flange, thickness_web, radius, sigma_yield, sigma_applied, c=1)`

## Please enter your name (daniel, fabian, felix, yannis)

In [1]:
name = 'yannis'

## Imports

In [2]:
import pandas as pd
import numpy as np
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *

In [3]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

## Constants

In [4]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}. Please verify.")

stringer_pitch = 200
effective_width = stringer_pitch/2

panel_element_length = 750/3
stringer_element_length = 750/3

# Hat stringer parameters
DIM1 = 25
DIM2 = 2
DIM3 = 20
DIM4 = 10




Your personal data is: sigma_yield = 490, EModulus = 65241.07. Please verify.


# Importing necessary files 

## Import everything for panels 

In [5]:
# Import panel properties and match the elements to the respective panel
paneldf = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')
panelPropertiesdf = pd.read_excel(f'../data/{name}/panel_properties.xlsx', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.rename(columns={'elements': 'Element ID'})
paneldf = paneldf.rename(columns={'Elements':'Element ID'})
paneldf = pd.merge(paneldf, element_component_df, on='Element ID', how='left', suffixes=('_caller', '_other'))
paneldf = pd.merge(paneldf, panelPropertiesdf, on='Element ID', how='left', suffixes=('_caller', '_other'))
paneldf.drop(columns=['FileID', 'Step', 'Layer'], inplace=True)
paneldf.head(5)

,Element ID,Loadcase,XX,XY,YY,Component Name,thickness,mass
0,1,1,-65.783310,25.897854,12.579947,panel1,4,0.00054
1,2,1,-81.867737,26.737642,20.594475,panel1,4,0.00054
2,3,1,-66.729820,36.480675,20.465578,panel1,4,0.00054
3,4,1,-96.377647,34.062004,2.383052,panel2,4,0.00054
4,5,1,-88.315163,26.684858,25.010015,panel2,4,0.00054


### We can already drop some columns from panel property

In [6]:
# Rearrange panel properties 
paneldf = paneldf.drop(columns=['YY','XY'])
paneldf.head(2)

,Element ID,Loadcase,XX,Component Name,thickness,mass
0,1,1,-65.783310,panel1,4,0.00054
1,2,1,-81.867737,panel1,4,0.00054


## Add a volume column to the panels 

In [7]:
paneldf['Volume'] = paneldf.apply(panel_element_volume, elementLength=panel_element_length, elementWidth=stringer_pitch, axis=1)
paneldf.head(3)

,Element ID,Loadcase,XX,Component Name,thickness,mass,Volume
0,1,1,-65.783310,panel1,4,0.00054,100000.0
1,2,1,-81.867737,panel1,4,0.00054,100000.0
2,3,1,-66.729820,panel1,4,0.00054,100000.0


## Import everything for stringers 

In [8]:
# Open and match stringer properties 
stringerdf = pd.read_csv(f'../data/{name}/stringer_v2.csv', index_col=0)
stringerdf = stringerdf.rename(columns={'Element Stresses (1D):CBAR/CBEAM Axial Stress':'XX'})
stringerdf = stringerdf.drop(['FileID', 'Step'], axis = 1)
stringerdf = stringerdf.reset_index()
stringerdf = stringerdf.rename(columns={'Elements': 'Element ID'})
stringerdf = pd.merge(stringerdf, element_component_df, on='Element ID', how='left', suffixes=('_caller', '_other'))
#print(stringerdf.to_string())

# Now add the stringer properties '../data/{name}/stringer_properties.csv
stringerPropertiesdf = pd.read_csv(f'../data/{name}/stringer_properties.csv', index_col=0)
stringerPropertiesdf = stringerPropertiesdf.reset_index()
stringerPropertiesdf.rename(columns={'beamsects': 'Component Name', 'beamsect_dim1': 'dim1', 'beamsect_dim2': 'dim2', 'beamsect_dim3': 'dim3', 'beamsect_dim4': 'dim4'}, inplace=True)
# Add "stringer" prefix to Component Name
stringerPropertiesdf['Component Name'] = 'stringer' + stringerPropertiesdf['Component Name'].astype(str)
# Merge the dataframes
stringerdf = pd.merge(stringerdf, stringerPropertiesdf, on='Component Name', how='left', suffixes=('_caller', '_other'))
stringerdf.head(5)

,Element ID,Loadcase,XX,Component Name,dim4,dim1,dim2,dim3
0,40,1,-76.643044,stringer1,15,25,2,20
1,41,1,-69.669487,stringer1,15,25,2,20
2,42,1,-78.626205,stringer1,15,25,2,20
3,43,1,-80.222389,stringer2,15,25,2,20
4,44,1,-84.115013,stringer2,15,25,2,20


## Add volume to the stringer elements

In [9]:
stringerdf['Volume']= stringerdf.apply(stringer_element_volume, elementLength = stringer_element_length, axis=1)
stringerdf.head(3)

,Element ID,Loadcase,XX,Component Name,dim4,dim1,dim2,dim3,Volume
0,40,1,-76.643044,stringer1,15,25,2,20,48000.0
1,41,1,-69.669487,stringer1,15,25,2,20,48000.0
2,42,1,-78.626205,stringer1,15,25,2,20,48000.0


# Now we rearrange the structure a bit

## First we split the 3 loadcases 

In [10]:
loadCase1dfPanel = paneldf[paneldf["Loadcase"] == 1]
loadCase2dfPanel = paneldf[paneldf["Loadcase"] == 2]
loadCase3dfPanel = paneldf[paneldf["Loadcase"] == 3]
loadCase1dfStringer = stringerdf[stringerdf["Loadcase"] == 1]
loadCase2dfStringer = stringerdf[stringerdf["Loadcase"] == 2]
loadCase3dfStringer = stringerdf[stringerdf["Loadcase"] == 3]
print(loadCase1dfPanel.head(5))
print(loadCase1dfStringer.head(5))

   Element ID  Loadcase         XX Component Name  thickness     mass  \
0           1         1 -65.783310         panel1          4  0.00054   
1           2         1 -81.867737         panel1          4  0.00054   
2           3         1 -66.729820         panel1          4  0.00054   
3           4         1 -96.377647         panel2          4  0.00054   
4           5         1 -88.315163         panel2          4  0.00054   

     Volume  
0  100000.0  
1  100000.0  
2  100000.0  
3  100000.0  
4  100000.0  
   Element ID  Loadcase         XX Component Name  dim4  dim1  dim2  dim3  \
0          40         1 -76.643044      stringer1    15    25     2    20   
1          41         1 -69.669487      stringer1    15    25     2    20   
2          42         1 -78.626205      stringer1    15    25     2    20   
3          43         1 -80.222389      stringer2    15    25     2    20   
4          44         1 -84.115013      stringer2    15    25     2    20   

    Volume  
0

# Now we need to combine the panels and the stringers

In [11]:
# For every loadcase
# Efficiently combine panels for load case 1 
panel_groups_1 = []
for i in range(1, 10):
    df1 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase1dfStringer[loadCase1dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_1.extend([df1, df2, df3])
lc1combined = pd.concat(panel_groups_1, ignore_index=True)
lc1combined.head(10)

,Element ID,Loadcase,XX,Component Name,thickness,mass,Volume,Stiffener,dim4,dim1,dim2,dim3
0,1,1,-65.783310,panel1,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
1,2,1,-81.867737,panel1,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
2,3,1,-66.729820,panel1,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
3,4,1,-96.377647,panel2,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
4,5,1,-88.315163,panel2,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
5,6,1,-95.011391,panel2,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
6,40,1,-76.643044,stringer1,NaN,NaN,48000.0,stiffener1,15.0,25.0,2.0,20.0
7,41,1,-69.669487,stringer1,NaN,NaN,48000.0,stiffener1,15.0,25.0,2.0,20.0
8,42,1,-78.626205,stringer1,NaN,NaN,48000.0,stiffener1,15.0,25.0,2.0,20.0
9,4,1,-96.377647,panel2,4.0,0.00054,100000.0,stiffener2,NaN,NaN,NaN,NaN


In [12]:
# Efficiently combine panels for load case 2
panel_groups_2 = []
for i in range(1, 10):
    df1 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase2dfStringer[loadCase2dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_2.extend([df1, df2, df3])
lc2combined = pd.concat(panel_groups_2, ignore_index=True)
lc2combined.head(10)

,Element ID,Loadcase,XX,Component Name,thickness,mass,Volume,Stiffener,dim4,dim1,dim2,dim3
0,1,2,-67.899490,panel1,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
1,2,2,-82.523071,panel1,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
2,3,2,-68.846001,panel1,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
3,4,2,-95.861076,panel2,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
4,5,2,-87.145088,panel2,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
5,6,2,-94.494827,panel2,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
6,40,2,-83.769035,stringer1,NaN,NaN,48000.0,stiffener1,15.0,25.0,2.0,20.0
7,41,2,-87.974434,stringer1,NaN,NaN,48000.0,stiffener1,15.0,25.0,2.0,20.0
8,42,2,-85.752190,stringer1,NaN,NaN,48000.0,stiffener1,15.0,25.0,2.0,20.0
9,4,2,-95.861076,panel2,4.0,0.00054,100000.0,stiffener2,NaN,NaN,NaN,NaN


In [13]:
# Efficiently combine panels for load case 3 
panel_groups_3 = []
for i in range(1, 10):
    df1 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase3dfStringer[loadCase3dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_3.extend([df1, df2, df3])
lc3combined = pd.concat(panel_groups_3, ignore_index=True)
lc3combined.head(10)

,Element ID,Loadcase,XX,Component Name,thickness,mass,Volume,Stiffener,dim4,dim1,dim2,dim3
0,1,3,-20.124739,panel1,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
1,2,3,-27.197670,panel1,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
2,3,3,-23.898079,panel1,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
3,4,3,-34.514473,panel2,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
4,5,3,-29.339600,panel2,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
5,6,3,-29.067791,panel2,4.0,0.00054,100000.0,stiffener1,NaN,NaN,NaN,NaN
6,40,3,-21.838360,stringer1,NaN,NaN,48000.0,stiffener1,15.0,25.0,2.0,20.0
7,41,3,-23.145233,stringer1,NaN,NaN,48000.0,stiffener1,15.0,25.0,2.0,20.0
8,42,3,-29.744377,stringer1,NaN,NaN,48000.0,stiffener1,15.0,25.0,2.0,20.0
9,4,3,-34.514473,panel2,4.0,0.00054,100000.0,stiffener2,NaN,NaN,NaN,NaN


In [14]:
# We fill the empty spaces with zero
lc1combined = lc1combined.fillna(0)
lc2combined = lc2combined.fillna(0)
lc3combined = lc3combined.fillna(0)
lc1combined

,Element ID,Loadcase,XX,Component Name,thickness,mass,Volume,Stiffener,dim4,dim1,dim2,dim3
0,1,1,-65.783310,panel1,4.0,0.00054,100000.0,stiffener1,0.0,0.0,0.0,0.0
1,2,1,-81.867737,panel1,4.0,0.00054,100000.0,stiffener1,0.0,0.0,0.0,0.0
2,3,1,-66.729820,panel1,4.0,0.00054,100000.0,stiffener1,0.0,0.0,0.0,0.0
3,4,1,-96.377647,panel2,4.0,0.00054,100000.0,stiffener1,0.0,0.0,0.0,0.0
4,5,1,-88.315163,panel2,4.0,0.00054,100000.0,stiffener1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
76,29,1,-81.867737,panel10,4.0,0.00054,100000.0,stiffener9,0.0,0.0,0.0,0.0
77,30,1,-65.783310,panel10,4.0,0.00054,100000.0,stiffener9,0.0,0.0,0.0,0.0
78,64,1,-78.626205,stringer9,0.0,0.00000,48000.0,stiffener9,15.0,25.0,2.0,20.0
79,65,1,-69.669487,stringer9,0.0,0.00000,48000.0,stiffener9,15.0,25.0,2.0,20.0


# Now we can aggregate the loadcases according to stiffeners

## Multiply volume and stress together for averaging 

In [15]:
lc1combined['XX * Volume'] = lc1combined['XX'] * lc1combined['Volume']
lc2combined['XX * Volume'] = lc2combined['XX'] * lc2combined['Volume']
lc3combined['XX * Volume'] = lc3combined['XX'] * lc3combined['Volume']
lc1combined.head(2)

,Element ID,Loadcase,XX,Component Name,thickness,mass,Volume,Stiffener,dim4,dim1,dim2,dim3,XX * Volume
0,1,1,-65.783310,panel1,4.0,0.00054,100000.0,stiffener1,0.0,0.0,0.0,0.0,-6.578331e+06
1,2,1,-81.867737,panel1,4.0,0.00054,100000.0,stiffener1,0.0,0.0,0.0,0.0,-8.186774e+06


## Load case 1 

In [16]:
lc1combined = lc1combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'thickness':'max',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc1combined['sigma_XX_avg'] = lc1combined['XX * Volume'] / lc1combined['Volume']
lc1combined = lc1combined.drop(['XX * Volume'], axis=1)
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg
Stiffener,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-80.921460
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-91.268052
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-90.983358
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.447716
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.222285
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.447716
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-90.983358
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-91.268052
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-80.921460


## Load case 2

In [17]:
lc2combined = lc2combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'thickness':'max',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc2combined['sigma_XX_avg'] = lc2combined['XX * Volume'] / lc2combined['Volume']
lc2combined = lc2combined.drop(['XX * Volume'], axis=1)
lc2combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg
Stiffener,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-83.382725
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-90.449854
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-90.715643
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.467101
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.280895
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.467101
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-90.715643
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-90.449854
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-83.382725


## Load case 3

In [18]:
lc3combined = lc3combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'thickness':'max',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc3combined['sigma_XX_avg'] = lc3combined['XX * Volume'] / lc3combined['Volume']
lc3combined = lc3combined.drop(['XX * Volume'], axis=1)
lc3combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg
Stiffener,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-26.883303
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-30.320594
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-30.226015
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-30.048066
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-29.973175
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-30.048066
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-30.226015
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-30.320594
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-26.883303


# Now we add Cross-Section Properties of the combined skin and hat stringer crosssection 

In [19]:
# Load case 1
lc1combined[['I_yy', 'areaTot', 'VolumeTot']] = lc1combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 2
lc2combined[['I_yy', 'areaTot', 'VolumeTot']] = lc2combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 3
lc3combined[['I_yy', 'areaTot', 'VolumeTot']] = lc3combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot
Stiffener,,,,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-80.921460,43392.340054,992.0,744000.0
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-91.268052,43392.340054,992.0,744000.0
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-90.983358,43392.340054,992.0,744000.0
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.447716,43392.340054,992.0,744000.0
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.222285,43392.340054,992.0,744000.0
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.447716,43392.340054,992.0,744000.0
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-90.983358,43392.340054,992.0,744000.0
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-91.268052,43392.340054,992.0,744000.0
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-80.921460,43392.340054,992.0,744000.0


# Now we calculate the columnbuckling with Euler Johnson 

In [20]:
# Load case 1EModulus, DIM1, DIM2, DIM3, sigma_yield, r
lc1combined['sigma_crip'] = lc1combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
# Load case 2
lc2combined['sigma_crip'] = lc2combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
#Load case 3
lc3combined['sigma_crip'] = lc3combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip
Stiffener,,,,,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-80.921460,43392.340054,992.0,744000.0,490
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-91.268052,43392.340054,992.0,744000.0,490
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-90.983358,43392.340054,992.0,744000.0,490
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.447716,43392.340054,992.0,744000.0,490
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.222285,43392.340054,992.0,744000.0,490
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.447716,43392.340054,992.0,744000.0,490
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-90.983358,43392.340054,992.0,744000.0,490
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-91.268052,43392.340054,992.0,744000.0,490
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-80.921460,43392.340054,992.0,744000.0,490


## Add lambda_crit to the loadcases

In [21]:
lc1combined['lambda_crit'] = lc1combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc2combined['lambda_crit'] = lc2combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc3combined['lambda_crit'] = lc3combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)

## Add lambda to the loadcases

In [22]:
lc1combined['lambda'] = lc1combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc2combined['lambda'] = lc2combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc3combined['lambda'] = lc3combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)

## Add radius of gyration to the loadcases

In [23]:
lc1combined['r_gyr'] = lc1combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc2combined['r_gyr'] = lc2combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc3combined['r_gyr'] = lc3combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)

## Now calculate the critical stress & Reserve Factor

In [24]:
lc1combined[['sigma_crit', 'Reserve Factor']] = lc1combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc2combined[['sigma_crit', 'Reserve Factor']] = lc2combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc3combined[['sigma_crit', 'Reserve Factor']] = lc3combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
Stiffener,,,,,,,,,,,,,,,,
stiffener1,744000.0,4.0,25.0,2.0,20.0,15.0,-80.921460,43392.340054,992.0,744000.0,490,51.265756,113.39935,6.613795,50.072548,0.412520
stiffener2,744000.0,4.0,25.0,2.0,20.0,15.0,-91.268052,43392.340054,992.0,744000.0,490,51.265756,113.39935,6.613795,50.072548,0.365754
stiffener3,744000.0,4.0,25.0,2.0,20.0,15.0,-90.983358,43392.340054,992.0,744000.0,490,51.265756,113.39935,6.613795,50.072548,0.366899
stiffener4,744000.0,4.0,25.0,2.0,20.0,15.0,-90.447716,43392.340054,992.0,744000.0,490,51.265756,113.39935,6.613795,50.072548,0.369072
stiffener5,744000.0,4.0,25.0,2.0,20.0,15.0,-90.222285,43392.340054,992.0,744000.0,490,51.265756,113.39935,6.613795,50.072548,0.369994
stiffener6,744000.0,4.0,25.0,2.0,20.0,15.0,-90.447716,43392.340054,992.0,744000.0,490,51.265756,113.39935,6.613795,50.072548,0.369072
stiffener7,744000.0,4.0,25.0,2.0,20.0,15.0,-90.983358,43392.340054,992.0,744000.0,490,51.265756,113.39935,6.613795,50.072548,0.366899
stiffener8,744000.0,4.0,25.0,2.0,20.0,15.0,-91.268052,43392.340054,992.0,744000.0,490,51.265756,113.39935,6.613795,50.072548,0.365754
stiffener9,744000.0,4.0,25.0,2.0,20.0,15.0,-80.921460,43392.340054,992.0,744000.0,490,51.265756,113.39935,6.613795,50.072548,0.412520


# Cleanup data for output 

## Drop unenessacry columns 

In [25]:
lc1combined = lc1combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc2combined = lc2combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc3combined = lc3combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc1combined

,sigma_XX_avg,I_yy,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
Stiffener,,,,,,,,
stiffener1,-80.921460,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.412520
stiffener2,-91.268052,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.365754
stiffener3,-90.983358,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.366899
stiffener4,-90.447716,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.369072
stiffener5,-90.222285,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.369994
stiffener6,-90.447716,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.369072
stiffener7,-90.983358,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.366899
stiffener8,-91.268052,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.365754
stiffener9,-80.921460,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.412520


## ROUND & Add together the load cases 

In [26]:
# Rename colums for concat
lc1combined = lc1combined.rename(columns={'sigma_XX_avg':'XX_avg_LC1', 'sigma_crit':'sigma_crit_LC1', 'Reserve Factor':'RF_LC1', 'sigma_crip':'sigma_crip_LC1'})
lc2combined = lc2combined.rename(columns={'sigma_XX_avg':'XX_avg_LC2', 'sigma_crit':'sigma_crit_LC2', 'Reserve Factor':'RF_LC2', 'sigma_crip':'sigma_crip_LC2'})
lc3combined = lc3combined.rename(columns={'sigma_XX_avg':'XX_avg_LC3', 'sigma_crit':'sigma_crit_LC3', 'Reserve Factor':'RF_LC3', 'sigma_crip':'sigma_crip_LC3'})

In [27]:
outputdf = pd.concat([lc1combined,lc2combined,lc3combined], axis = 1)
outputdf = outputdf.round(rounding_digits)
# After concatenation, keep only the first column of cross section propertries and drop the rest
outputdf['I_y'] = outputdf.filter(like='I_yy').iloc[:, 0]  # Take the first I_yy column
outputdf['Lambda'] = outputdf.filter(like='lambda').iloc[:, 1]  # Take the first lambda column
outputdf['Lambda_crit'] = outputdf.filter(like='lambda_crit').iloc[:, 0]  # Take the first I_yy column
outputdf['R_gyr'] = outputdf.filter(like='r_gyr').iloc[:, 0]  # Take the first I_yy column
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('I_yy')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('lambda')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('r_gyr')])
outputdf.head(10)

,XX_avg_LC1,sigma_crip_LC1,sigma_crit_LC1,RF_LC1,XX_avg_LC2,sigma_crip_LC2,sigma_crit_LC2,RF_LC2,XX_avg_LC3,sigma_crip_LC3,sigma_crit_LC3,RF_LC3,I_y,Lambda,Lambda_crit,R_gyr
Stiffener,,,,,,,,,,,,,,,,
stiffener1,-80.9215,490,50.0725,0.4125,-83.3827,490,50.0725,0.4003,-26.8833,490,50.0725,1.2417,43392.3401,113.3993,51.2658,6.6138
stiffener2,-91.2681,490,50.0725,0.3658,-90.4499,490,50.0725,0.3691,-30.3206,490,50.0725,1.1010,43392.3401,113.3993,51.2658,6.6138
stiffener3,-90.9834,490,50.0725,0.3669,-90.7156,490,50.0725,0.3680,-30.2260,490,50.0725,1.1044,43392.3401,113.3993,51.2658,6.6138
stiffener4,-90.4477,490,50.0725,0.3691,-90.4671,490,50.0725,0.3690,-30.0481,490,50.0725,1.1109,43392.3401,113.3993,51.2658,6.6138
stiffener5,-90.2223,490,50.0725,0.3700,-90.2809,490,50.0725,0.3698,-29.9732,490,50.0725,1.1137,43392.3401,113.3993,51.2658,6.6138
stiffener6,-90.4477,490,50.0725,0.3691,-90.4671,490,50.0725,0.3690,-30.0481,490,50.0725,1.1109,43392.3401,113.3993,51.2658,6.6138
stiffener7,-90.9834,490,50.0725,0.3669,-90.7156,490,50.0725,0.3680,-30.2260,490,50.0725,1.1044,43392.3401,113.3993,51.2658,6.6138
stiffener8,-91.2681,490,50.0725,0.3658,-90.4499,490,50.0725,0.3691,-30.3206,490,50.0725,1.1010,43392.3401,113.3993,51.2658,6.6138
stiffener9,-80.9215,490,50.0725,0.4125,-83.3827,490,50.0725,0.4003,-26.8833,490,50.0725,1.2417,43392.3401,113.3993,51.2658,6.6138


# Generate output file 

In [28]:
outputdf.to_excel(f'../data/{name}/output/processed_f.xlsx')